In [2]:
'''
I. 추천 시스템의 개요와 배경
    - 많은 플랫폼과 기업들이 추천시스템을 적용 후 매출이 큰 폭으로 향상
    - 추천 시스템은 크게 콘텐츠 기반 필터링 방식과 협업 필터링 방식으로 나뉨, 협업 필터링 방식은 다시 최근접 이웃 협업 필터링과 잠재 요인 협업 필터링으로 나뉨
    - 행렬 분해 기법을 이용한 잠재 요인 협업 필터링 방식이 넷플릭스 추천 시스템 경연대회에서 우승하면서 많은 플랫폼이 해당방식의 추천 시스템 사용

II. 콘텐츠 기반 필터링 추천 시스템
    - 사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
    
III. 최근접 이웃 협업 필터링
    - 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 방식이 '협업 필터링'
    - 사용자(행)-아이템(열) 평점(값) 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가
    - 사용자-아이템 평점의 경우 행에 사용자 정보, 열에 아이템 정보, 평점이 값에 위치해야 하므로 사용자,아이템,평점이 각 열에 위치한 경우 pandas의 pivot_table()등의 함수를 이용해 형태를 바꿔줘야 함
    - 사용자-아이템 평점 행렬은 많은 아이템을 열로 가지는 다차원 행렬 + 희소행렬의 특징을 지님
    - 최근접 이웃 협업 필터링(=메모리 협업 필터링)은 일반적으로 사용자 기반과 아이템 기반으로 다시 나뉨(사용자 기반 : 당신과 비슷한 고객들이 다음 상품도 구매했습니다, 아이템 기반 : 이 아이템을 구마핸 다른 고객들은 다음 상품도 구매했습니다.)
    - 사용자 기반은 행이 사용자, 열이 아이템일 때 평점 패턴이 유사한 사용자에게 같은 아이템을 추천하는 것
    - 아이템 기반은 행이 아이템, 열이 사용자 일때, 평점 패턴이 유사한 아이템을 사용자에게 추천하는 것
    - 일반적으로 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 높음 -> 추천 시스템에 사용되는 데이터는 희소행렬이기 때문에 유사도 측정을 위해 주로 코사인 유사도를 이용

IV. 잠재 요인 협업 필터링
    - 대규모 다타원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출하는데, 이러한 기법을 행렬 분해라고 함
    - 다차원 희소행렬인 사용자-아이템 행렬 데이터를 밀집행렬의 사용자-잠재요인 행렬과 아이템-잠재요인 행렬의 전치행렬로 분해 후 두 행렬을 내적해서 새로운 예측 사용자-아이템 평점 행렬데이터를 만드는 알고리즘
    - 행렬 분해에 의해 추출되는 잠재요인은 명확히 정의할 수 없음(해당 상황에 따른 특성값으로 정의)
        ex) 사용자-영화 평점 행렬 R에서 R(u,i)는 u번째 사용자의 i번째 영화에 대한 평점
            사용자-잠재요인 행렬을 P라고 하고 P의 column이 factor1(액션영화 선호도), factor2(로맨스영화 선호도)일 때, P(u,k)는 u번째 사용자의 k 번째 factor에 대한 ㅓㄴ호도
            아이템 - 잠재요인 행렬을 Q라고 할 때, Q(i,k)는 i번째 영화에 대한 k의 특징값
            이 때 사용자의 평점 예측값 R^ = P.Q^T로 구할 수 있음
    
    1. 행렬 분해의 이해
        - 행렬분해는 다차원의 매트릭스를 저차원 매트릭스로 분해하는 기법으로서 대표적으로 SVD(Singular Vector Decomposition), NMF(Non-Negative Matrix Factorization) 등이 존재
        - 사용자-아이템 행렬 R(M x N)을 사용자-K차원 잠재요인행렬 P(M x K)과 K차원 잠재요인-아이템행렬 Q.T(K x N)으로 분해
        - R의 u행 사용자와 i열 아이템에 위치한 평점을 r(u,i)라고 하면 r(u,i) = p_u * q.T_i
        - 여기서 행렬분해는 널 값이 없는 행렬에 적용가능한 SVD에 확률적 경사하강법(SGD)이나 ALS(Alternating Least Squares)의 방식을 이용한 방식을 이용
    
    2. 확률적 경사 하강법을 이용한 행렬 분해
        - P와 Q 행렬로 계산된 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추
        - 실제값과 예측값의 오류 최소화와 L2규제를 고려한 비용함수식 : Min ∑(r(u,i) - p_u * q.T_i)² + λ(||q_i||²+||p_u||²)
        - 위의 비용함수를 최소화하기 위해서 새롭게 업데이트 되는 p'_u와 q'_i는 각각 p'_u = p_u + n(e_u,i * q_i - λ*p_u), q'_i = q_i + n(e_u,i * p_u - λ*q_i) ; n은 SGD 학습률, λ은 L2규제 계수, e_u,i는 실제 행렬값과 예측행렬 값의 차이(r_u,i - r_hat_u,i)
'''
# SGD 행렬 분해 예제
# 잠재 요인은 3차원으로 설정
import numpy as np

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬을 임의로 설정
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users,K))
Q = np.random.normal(scale=1./K, size=(num_items,K))

In [7]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 get_rmse()함수를 생성
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [8]:
# SGD 행렬 분해 수행
# steps는 SGD 반복 수행 횟수, learning_rate는 SGD 학습률, r_lambda는 L2 Regularization 계수

# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.18918148042182
### iteration step :  50  rmse :  0.4540281501803243
### iteration step :  100  rmse :  0.15115898740988784
### iteration step :  150  rmse :  0.0726996577447477
### iteration step :  200  rmse :  0.04246278530171513
### iteration step :  250  rmse :  0.028877999722529146
### iteration step :  300  rmse :  0.02244555046880699
### iteration step :  350  rmse :  0.019411195442597978
### iteration step :  400  rmse :  0.017983693814787764
### iteration step :  450  rmse :  0.017300739181608172
### iteration step :  500  rmse :  0.01696414305881988
### iteration step :  550  rmse :  0.016791808565876604
### iteration step :  600  rmse :  0.01669850293306415
### iteration step :  650  rmse :  0.016642938601229156
### iteration step :  700  rmse :  0.016604553140343155
### iteration step :  750  rmse :  0.016572993477699336
### iteration step :  800  rmse :  0.016543116786402502
### iteration step :  850  rmse :  0.016512497103841964
### iter

In [10]:
# 분해된 P와 Q ㅎㅁ수를 P*Q.T로 예측 행렬을 만들어서 출력
print(np.round(np.dot(P, Q.T),3))

[[3.991 0.897 1.306 2.002 1.663]
 [6.695 4.978 0.978 2.981 1.003]
 [6.676 0.392 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.141]]


In [13]:
'''
IV. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
    - TMDB는 영화 데이터 사이트인 IMDB의 주요 영화 5000개 영화에 대한 메타정보를 가공 후 캐글에서 제공
    - https://www.kaggle.com/tmdb/tmdb-movie-metadata 에서 tmdb_5000_credit.csv, tmdb_5000_movies.csv 다운
    - 영화를 구성하는 다양한 컨텐츠를 기반으로 이들의 유사성을 통해 영화를 추천하는 콘텐츠 기반 필터링 실습
'''
# 데이터 로딩 및 가공
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [14]:
# 콘텐츠 기반 필터링에 사용할 칼럼으로 DataFrame 재구성
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count','popularity', 'keywords', 'overview']]

In [15]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1] # genre와 keywords 같은 경우 한 칼럼에 여러개의 데이터가 존재

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [19]:
# 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출 
# ast 모듈의 literal_eval()함수를 이용해서 문자열을 객체로 변환
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

movies_df[['genres','keywords']][:1]
# 이제 칼럼에 들은 값이 단순한 문자열이 아닌 실제 딕셔너리의 기능을 하게 됨

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


In [26]:
# genres,keywords 칼럼에서 장르명, 키워드 값만 리스트 객체로 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [27]:
# 장르 콘텐츠 유사도 측정
# 한 영화는 여러개의 장르로 구성되어 있으므로 장르를 문자열로 변경한 뒤 피처벡터화 해서 코사인 유사도를 이용해 장르 유사도를 측정할 수 있음

from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,1))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 22)


In [33]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:3])

(4803, 4803)
[[1.         0.77459667 0.51639778 ... 0.         0.         0.        ]
 [0.77459667 1.         0.66666667 ... 0.         0.         0.        ]
 [0.51639778 0.66666667 1.         ... 0.         0.         0.        ]]


In [34]:
# movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출하기 위해 argsort()[:, ::-1]를 이용
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:3])

[[   0 3494   46 ... 3038 3037 2401]
 [  19  315   38 ... 3069 3067 2401]
 [1740    2 1542 ... 3000 2999 2401]]


In [37]:
genre_sim[1,19] #2행에서 자기자신(1)이 안나오는 것은 아마 부동소수점 문제로 코사인유사도가 1보다 높은 경우가 발생하기 때문인듯

1.0000000000000002

In [109]:
# 장르 콘텐츠 필터링을 이용한 영화 추천

# 장르 유사도에 따라 영화를 추천하는 함수 생성
def find_sim_movie(df, sorted_ind, title_name, top_n=10): #title_name은 곡객이 선정한 추천 기준이 되는 영화제목, sorted_ind는 레코드별 장르 코사인 유사도 인덱스
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    if title_index in similar_indexes: # 추천 기준은 삭제
        similar_indexes = np.delete(similar_indexes, np.where(similar_indexes == title_index))
        similar_indexes = np.append(similar_indexes, sorted_ind[title_index, (top_n):(top_n+1)])
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [112]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[ 883 2801 4065 3112 2072 1370 1149 2582 3012 1847]


,title,vote_average
883,Catch Me If You Can,7.7
2801,The Funeral,7.3
4065,Mi America,0.0
3112,Blood Done Sign My Name,6.0
2072,Freedom Writers,7.5
1370,21,6.5
1149,American Hustle,6.8
2582,The Place Beyond the Pines,6.8
3012,The Outsiders,6.9
1847,GoodFellas,8.2


In [114]:
# 평점과 관객수 등을 고려해서 개선해보자면 
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

#위와 같이 평점은 소수에 의해 왜곡 받을 수 있다는 점을 고려, 가중치가 부여된 평점을 이용
# 가중 평점  = (v/(v+m)) * R + (m/(v+m)) * C, v는 개별 영화에 평점을 투표한 횟수, m은 평점을 부여하기 위한 최소 투표 횟수, R은 개별 양화에 대한 평균 평점, C는 전체 영화에 대한 평균 평점
# v는 movies_df의 'vote_count', R은 ' vote_average', m 은 직접 결정해야되는데 여기서는 전체 투표 횟수의 상위 60%로 설정

C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(round(C,3), round(m,3))


6.092 370.2


In [115]:
# 가중 평점 함수 생성
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]


,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [116]:
# 새롭게 정의된 평점 기준에 따라 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정 뒤 weight_vote 칼럼이 노은 순으로 top_n만큼 추출
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
2072,Freedom Writers,7.5,6.782322
1149,American Hustle,6.8,6.717525
2582,The Place Beyond the Pines,6.8,6.670483
3012,The Outsiders,6.9,6.447704
1370,21,6.5,6.413490


In [123]:
'''
VI. 아이템 기반 최근접 이웃 협업 필터링 실습
    - 사용자-영화 평점 행렬 데이터 세트를 이용해 아이템 기반의 협업 필터링 구현
    - https://grouplens.org/datasets/movielens/latest/에서 ml-latest-samll.zip 다운
'''
# 데이터 가공 및 변환
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [124]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [126]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [127]:
# ratings 데이터를 사용자-아이템 방식으로 바꾸기 -> pivot_table()함수 이용
ratings - ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')
ratings_matrix.head() #희소행렬

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# movieId 대신 movies 데이터셋에서 title을 가져오고 NaN은 0점으로 변환

#title 칼럼을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings, movies, on='movieId')

#columns = 'title'로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')

#NaN값을 0으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
# 영화간 유사도 산출
# 이전과 같이 sklearn의 cosine_similarity()를 사용하지만, cosine_similarity()는 행기준이기 때문에 지금 이상태로 사용하면 영화간 유사도가 아닌 유저간 유사도가 생성됨
# pandas의 transpose()함수를 이용, 열과 행을 바꿔짐
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [134]:
# item_sim을 DataFrame으로 변환한 item_sim_df를 이용, 대부와 유사도가 높은 상위 6개 종목 추출
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [136]:
# 인셉션과 유사도가 높은 종목 추출(자기자신 제외)
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

In [137]:
'''
    2. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
        - 개인이 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기바능로 해 새롭게 모든 영화으 예측 평점 데이터를 계산 후 높은 예측 평점을 가진 영화를 추천
        - 예측 평점 식 : R_hat_u,i = ∑(->N)(S_i,N * R_u,N)/∑(->N)(|S_i,N|), R_hat_u,i : 사용자 u, 아이템 i의 개인화된 예측 평점 값, S_i,N : 아이템 i와 가장 유사도가 높은 Top-N개 아이템의 유사도 벡터, R_u,N : 사용자 u의 아이템 i와 가장 유사도가 높은 Top-N개 아이템에 대한 실제 평점 벡터       
'''
# 인자로 사용자-영화 평점 넘파이 행렬과 영화 간의 유사도를 가지는 넘파이 행렬을 받아 예측 평점을 계산하는 함수 생성
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# 실제로 계산해보면
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


In [138]:
# 예측평점과 실제평점의 차이를 구해보면
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


In [139]:
# 예측률 개선을 위해 특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수로 predict_rating함수를 수정하면
# 데이터가 클 경우 시간이 오래걸림에 주의
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [140]:
# predict_rating_topsim()함수를 이용해 예측평점과 MSE를 구해보면
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  3.6949696289030554


In [141]:
# 특정 사용자에게 영화추천을 해보면 ex) userId = 9인 사람에게 영화추천
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10] #영화 평점 목록 확인

title
Adaptation (2002)                                                                 5.0
Austin Powers in Goldmember (2002)                                                5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Producers, The (1968)                                                             5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Back to the Future (1985)                                                         5.0
Glengarry Glen Ross (1992)                                                        4.0
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)                                     4.0
Name: 9, dtype: float64

In [9]:
# 이미 본 영화를 추천하지 않도록 이미 본 영화를 제외하는 함수를 생성
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [10]:
#예측 평점 DataFrame과 추천하려는 userId, 추천 후보 영화 리스트, 추천 상위 영화 개수를 인자로 받아서 
#가장 높은 예측 평점을 가진 영화를 반환하는 함수 생성
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


In [1]:
'''
VII. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습
    - SGD 기반의 행렬분해 구현, 이를 기반으로 사용자에게 영화 추천
'''
# 행렬분해 예제에서 했던 get_rmse() 코드를 활용, 행렬 분해 로직 함수 생성
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros): # R은 사용자-아이템 평점 행렬
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01): # R은 사용자-아이템 평점 행렬, K는 잠재요인 차원의 수, steps는 SGD 반복횟수, r_lambda 는 L2 규제 계수
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [2]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId', columns='movieId')

#title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')
#columns = 'title'로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId',columns='title')

In [5]:
# 만들어진 사용자-아이템 평점 행렬을 matrix_factorization()함수를 이용해 분해
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate = 0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.2068545530233154
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720403
### iteration step :  100  rmse :  0.177429275272091
### iteration step :  110  rmse :  0.1716522696470749
### iteration step :  120  rmse :  0.16695181946871726
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679643
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715425
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  rmse

In [8]:
# 반환된 예측 행렬을 영화 타이틀을 칼럼으로 하는 DataFrame으로 변환
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


In [11]:
# 개인 사용자에게 영화 추천
unseen_list = get_unseen_movies(ratings_matrix,9) # 9번 사용자에 대해 영화 추천

recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


In [5]:
'''
VIII. 파이썬 추천 시스템 패키지 - Surprise
    1. Surprise 패키지 소개
        - Surprise는 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공하는 추천 시스템 구축 전용 패키지
        - pip install scikit-surprise 또는 conda install -c conda-forge scikit-surprise 로 설치/ Visual Studio Build Tools 2015 이상 버전 필요(설치필요-1장2절참고)
        - 사용자/아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반 잠재요인 협업 필터링 등을 쉽게 적용, 구사 가능
        - 자세한 내용은 https://surprise.readthedocs.io/en/stable/ 참고
    
    2. Surprise를 이용한 추천 시스템 구축
        - 추천 데이터를 학습용/테스트용으로 분리 후 SVD 해열ㄹ 분해를 통한 잠재요인 협업 필터링 수행
'''
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Surprise는 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트를 가져오는 API 제공
# Surprise Dataset 클래스의 load_builtin()은 무비렌즈 사이트에서 제공하는 데이터를 받아들이는 함수; ml-100k는 10만개 평점 데이터, ml-1m은 100만개 평점 데이터
data = Dataset.load_builtin('ml-100k') # 데이터를 다운 받은 후 로컬 디렉토리에 저장(C:\Users\1/.surprise_data/ml-100k)

#데이터 분리
trainset, testset = train_test_split(data, test_size=0.25,random_state=0)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\1/.surprise_data/ml-100k


In [8]:
# Surprise는 자체적으로 pivot_table()로 변환하기 때문에 로우 타입의 데이터를 그대로 사용

# SVD로 잠재 요인 협업 필터링 수행
algo = SVD()
algo.fit(trainset)

In [9]:
# Surprise에서 추천을 예측하는 메서드는 test()와 predict() 두개로, 
# test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하고
# predict()는 개별 사용자와 영화에 대한 추천 평점을 반환

predictions = algo.test(testset)
print(type(predictions), len(predictions))
predictions[:5] # r_ui는 실제평점, est_3은 예츠 평점, was_imposile이 True이면 예측값을 생성할 수 없는 데이터라는 의미

<class 'list'> 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.776882363877712, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6695346152924175, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9422273720846555, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4829533610803325, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.264587963672355, details={'was_impossible': False})]

In [11]:
# uid, iid 등의 속성에 접근하려면
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.776882363877712),
 ('882', '291', 3.6695346152924175),
 ('535', '507', 3.9422273720846555)]

In [14]:
# predict()를 이용하려면
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.18   {'was_impossible': False}


In [15]:
# 에측평점과 실제 평점간의 차이를 구해보면
accuracy.rmse(predictions)

RMSE: 0.9486


0.9485501049408018

In [17]:
'''
    내장 데이터 파일이 아닌 다른 데이터 파일으 로딩해 추천시스템을 구혀하기전 Surprise 패키지를 구성하는 모듈을 알아보면
    3. Surprise 주요 모듈 소개
        (i) Dataset 
            - Surprise는 첫번째 칼럼을 사용자 아이디, 두번째 칼럼을 아이템 아이디, 세번째 칼럼을 평점으로 가정해 데이터를 로딩하고 네번째 칼럼붙는 로딩을 수행하지 않음
            - 따라서 일반 DataFrame 등을 사용할 때도 순서가 사용자-아이템-평점 이어야 함
            - Dataset.load_from_df(df, reader) : 판다스 DF를 로딩할 때 사용, reader로 파일의 포맷을 지정
            - Dataset.load_from_file(file_path, reader) : 일반 OS에서 파일을 로딩할 때 사용(콤마, 탭 등으로 칼럼이 분리된 포맷을 로딩), reader로 파일의 포맷 지정
        
        (ii) OS 파일을 Surprise 데이터 세트로 로딩
            - Surprise에 OS 파일을 로딩할 때 주의할 점은 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서 안됨 -> DataFrame의 to_csv() 함수를 이용해 칼럼 헤더 삭제
'''
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings.to_csv('ratings_noh.csv',index=False, header=False) # to_excel과 같은 원리, 저장 옵션을 header = False로 지정해주면서 header를 없애는 것

from surprise import Reader
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5)) 
# Reader 클래스를 이용해 로딩할 데이터의 특성을 입력해 줘야함
# line_format은 칼럼의 이름을 순서대로 나열(공백으로 문자열 분리 후 칼럼으로 인식), sep 은 분리자, 디폴트는 '\t', rating_scale은 평점값의 범위

data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

In [18]:
# 데잍 ㅓ분리
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [19]:
'''
        (iii) 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
            - DataFrame 역시 사용자 아이디, 아이템아이디, 평점 칼럼 순서를 지켜야 함
'''
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [21]:
'''
    4. Surprise 추천 알고리즘 클래스
        - SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
        - KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
        - BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
        - 이 외 SVD++, NMF 등 -> http://surprise.readthedocs.io/rn/stable/prediction_algorithms_package.html 참고
        
        _ SVD 클래스의 입력 파라미터는
            (i) n_factors : 잠재 요인 K의 개수, 디폴트는 100, 커질수록 정확도가 높아질 수 있으나 과적합 야기 가능
            (ii) n_epochs : SGD 수행시 반복횟수, 디폴트 20
            (iii) biased (bool) : 베이스라인 사용자 편향 적용 여부, 디폴트 True
        
        - 알고리즘의 예측 성능 벤치마크 결과는 http://surpriselib.com/에서 확인(시간이 많이 걸리는 SVD++를 제외하면 SVD와 k-NN Baseline이 성능이 좋음)
    
    5. 베이스라인 평점이란
        - 영화나 상품의 각 평가가 각 개인의 성향에 따라 달라질 수 있다는 점을 고려(전반적으로 후한 평가를 주는 사람 vs 냉정한 평가를 하는 사람)
        - 이러한 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점이라고 함
        - 베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수(= 사용자별 아이템 평점 평균 값 - 전체 평균 평점) + 아이템 편향 점수(= 아이템별 평점 평균 값 - 전체 평균 평점)
        - ex) 모든 사용자의 모든 영화에 대한 평균 평점이 3.5, 어벤저스의 평균 평점이 4.2, 사용자 A의 평균 평점이 3.0일 때
              A의 어벤저스에 대한 예측 평점 = 3.5 + (3.0 - 3.5) + (4.2 - 3.5) = 3.7
'''
# 교차 검증과 하이퍼 파라미터 튜닝 - cross_validate() 와 GridSearchCV 이용

# 교차 검증
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8764  0.8722  0.8765  0.8708  0.8750  0.8742  0.0023  
MAE (testset)     0.6720  0.6691  0.6733  0.6705  0.6758  0.6721  0.0023  
Fit time          9.60    10.41   10.31   11.86   9.90    10.42   0.78    
Test time         0.23    0.34    0.47    0.41    0.26    0.34    0.09    


{'test_rmse': array([0.87643577, 0.87218345, 0.87647166, 0.87075416, 0.87497533]),
 'test_mae': array([0.67203114, 0.66906142, 0.67326283, 0.67047263, 0.67581895]),
 'fit_time': (9.603499174118042,
  10.414035558700562,
  10.30609679222107,
  11.857207775115967,
  9.897334575653076),
 'test_time': (0.23286724090576172,
  0.34180521965026855,
  0.4737269878387451,
  0.4067661762237549,
  0.2608504295349121)}

In [22]:
# 하이퍼 파라미터 튜닝 - SVD는 주로 n_epochs와 n_factors를 튜닝
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8779324950966595
{'n_epochs': 20, 'n_factors': 50}


In [23]:
# Surprise를 이용한 개인화 영화 추천 시스템 구축

# 이제는 Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천
# 이를 위해서 ratings.csv 전체를 학습 데이터로 사용해야 함 -> train_test_split을 거치치 않고 이를 바로 fit()을 통해 학습시키려 하면 오류 발생
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

# 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스 사용

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [24]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [26]:
# 학습
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

uid = str(9)
iid = str(42)

# 9번 사용자의 42번 아이템에 대한 예측 평점 출력
pred = algo.predict(uid, iid, verbose=True)



사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama
user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
# 이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천

def get_unseen_surprise(ratings, movies, userId): # Surprise 패키지용 함수
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10): # Surprise 패키지용 함수
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
